In [0]:
# Li
library(dataiku)
library(rpart)
# Recipe inputs
base_train <- dkuReadDataset("base_train", samplingMethod="head", nbRows=100000)

In [0]:
# Recipe outputs
base_scm_classification_min_model <- dkuManagedFolderPath("8jrmex16")